#Money Tree Map
####Using QGIS and Saint Louis Open Data to map tree features against City Block median housing prices

###

##Importing Data

In [1]:
!pip install geopandas
import geopandas as gp

In [2]:
tdf = gp.read_file('CITY_TREES.dbf')

In [3]:
print(tdf.columns)
print(tdf.shape)

Index(['ON_', 'STREET_NUM', 'STREET', 'COMMON', 'DBH', 'STEMS', 'CONDITION',
       'STORM_DAMA', 'HARDSCAPE_', 'OVERHEAD_U', 'XCOORD', 'YCOORD', 'GUID',
       'ADDRESS', 'ELIGIBLE', 'REPLANT', 'TREATED', 'LOCATION_T', 'LOCATION'],
      dtype='object')
(118688, 19)


###Nan values

In [7]:

# c = tdf[tdf['LOCATION_T'].isna()]
c = tdf[tdf['COMMON'].isna()]

c.head()

,ON_,STREET_NUM,STREET,COMMON,DBH,STEMS,CONDITION,STORM_DAMA,HARDSCAPE_,OVERHEAD_U,XCOORD,YCOORD,GUID,ADDRESS,ELIGIBLE,REPLANT,TREATED,LOCATION_T,LOCATION
118584,None,4163,TYROLEAN AVE,None,0,0,N/A,None,None,None,0.0,0.0,67FEB08E-A3F9-49A6-873F-972D27492B96,4163 TYROLEAN AVE,Yes,"Maple, Paperbark",None,Easement Tree,4163 TYROLEAN AVE
118585,None,5638,ROSA AVE,None,0,0,N/A,None,None,None,0.0,0.0,6CE21E6E-CD0A-454F-9678-074A4A4EEA75,5638 ROSA AVE,Yes,"Maple, Sugar 'Green Mountain'",None,Easement Tree,5638 ROSA AVE
118587,None,4359,HOLLY HILLS BLVD MEDIAN,None,0,0,N/A,None,None,None,0.0,0.0,9E0EFB29-8237-41F5-912C-09347A0D8A41,4359 HOLLY HILLS BLVD MEDIAN,Yes,"Zelkova, 'Village Green'",None,Easement Tree,4359 HOLLY HILLS BLVD MEDIAN
118590,None,4444,PAGE BLVD,None,0,0,N/A,None,None,None,0.0,0.0,34966B7E-C315-4AD3-9787-EB2E12D5ACCF,4444 PAGE BLVD,Yes,Kentucky Coffeetree,None,Easement Tree,4444 PAGE BLVD
118591,None,4444,PAGE BLVD,None,0,0,N/A,None,None,None,0.0,0.0,0A99B247-F8AE-4F58-9356-1575E1BE9E1B,4444 PAGE BLVD,Yes,Kentucky Coffeetree,None,Easement Tree,4444 PAGE BLVD


##Cleaning Tree Data

In [8]:
tdf.head()

,ON_,STREET_NUM,STREET,COMMON,DBH,STEMS,CONDITION,STORM_DAMA,HARDSCAPE_,OVERHEAD_U,XCOORD,YCOORD,GUID,ADDRESS,ELIGIBLE,REPLANT,TREATED,LOCATION_T,LOCATION
0,OAK HILL AVE,4067,UTAH ST,"Maple, red",5,1,Fair,No,No,No,889983.173256,1.006871e+06,{000025D7-9880-4FEA-9418-289F5E506BE8},4067 UTAH ST,Yes,None,FALSE,Easement Tree,4067 UTAH ST
1,SWAN AVE,4427,SWAN AVE,"Maple, silver (restricted use)",18,1,Poor,No,No,No,568477.616427,1.016695e+06,{00004E14-7BFA-4BFA-AE16-B97C20D6C373},4427 SWAN AVE,Yes,None,FALSE,Easement Tree,4427 SWAN AVE
2,ENRIGHT AVE,5501,ENRIGHT AVE,Vacant,0,0,N/A,No,No,No,882794.969857,1.027355e+06,{00005BB3-56A7-4DE0-8830-2BD5FF695DDE},5501 ENRIGHT AVE,Yes,None,FALSE,Easement Tree,5501 ENRIGHT AVE
3,BLOW ST,3655,BLOW ST,Vacant,0,0,N/A,No,No,No,887255.794446,9.921068e+05,{000085FC-BD37-4D9D-BBF6-0E3287270D68},3655 BLOW ST,Yes,"Redbud, Eastern",FALSE,Easement Tree,3655 BLOW ST
4,COLEMAN AVE,1821,COLEMAN AVE,Vacant,0,0,N/A,No,No,No,899440.025369,1.025467e+06,{0000A65A-A225-4A9C-9FBA-213CA68E6D5F},1821 COLEMAN AVE,Yes,None,FALSE,Easement Tree,1821 COLEMAN AVE


###Ash tree cleaning

In [ ]:
ash = tdf[tdf['COMMON'].str.contains('[Aa]sh', na = False)]
ash = ash['COMMON'].str.capitalize()
ash.unique()


array(['Ash, green', 'Ash, white', 'Ash, pop', 'Mountainash, american',
       'Ash, blue', 'Ash green', 'Ash, european', 'Hawthorn, washington',
       'Ash, spp.', 'Green ash', 'Ash, narrow-leafed', 'Ash, black',
       'Ash'], dtype=object)

###General tree cleaning
Here we are going to apply the same sort of species cleaning for the other genera of trees.

In [45]:
# tdf.head()
# tdf['COMMON'].unique()
genus = ["[Aa]sh", 'Beech', 'Birch', 'Cedar', 'Cherry', 'Fir', 'Ginkgo', 'Hawthorn',
         'Holly', '[Oo]ak', '[Pp]ine', 'Poplar', 'Redwood', 'Spruce', 'Willow', 'Yew', 'Cottonwood',
         'Dogwood', 'Chestnut', 'Pear', 'Apricot', 'Magnolia', 'Walnut', 'Crabapple',
         'Persimmon', 'Tulip', 'Larch', 'Linden', 'Lilac', 'Osage', 'Sycamore', 'Hornbeam',
         'Blackgum', 'Hackberry', 'Sweetgum','Elm', 'Arbor', 'Hickory', 'Cherry',
         'Serviceberry', 'Maple', 'Vacant', 'Redbud', 'Zelkova', '[Cc]ypress',
         'Mulberry']
tdf = tdf[tdf['COMMON'].notna()]
for g in genus:
  for i in tdf.index:
    if g in tdf.loc[i, 'COMMON']:
      tdf.loc[i, 'COMMON'] = g


In [46]:
# samp = tdf.copy()
# samp = samp.sample(1000)
# genus = ["[Aa]sh", 'Beech', 'Birch', 'Cedar', 'Cherry', 'Fir', 'Ginkgo', 'Hawthorn',
#          'Holly', '[Oo]ak', '[Pp]ine', 'Poplar', 'Redwood', 'Spruce', 'Willow', 'Yew', 'Cottonwood',
#          'Dogwood', 'Chestnut', 'Pear', 'Apricot', 'Magnolia', 'Walnut', 'Crabapple',
#          'Persimmon', 'Tulip', 'Larch', 'Linden', 'Lilac', 'Osage', 'Sycamore', 'Hornbeam',
#          'Blackgum', 'Hackberry', 'Sweetgum','Elm', 'Arbor', 'Hickory', 'Cherry',
#          'Serviceberry', 'Maple', 'Vacant', 'Redbud', 'Zelkova', '[Cc]ypress',
#          'Mulberry']
# samp = samp[samp['COMMON'].notna()]

# for g in genus:
#   for i in samp.index:
#     if g in samp.loc[i,'COMMON']:
#       samp.loc[i, 'COMMON'] = g
tdf.head(50)

,ON_,STREET_NUM,STREET,COMMON,DBH,STEMS,CONDITION,STORM_DAMA,HARDSCAPE_,OVERHEAD_U,XCOORD,YCOORD,GUID,ADDRESS,ELIGIBLE,REPLANT,TREATED,LOCATION_T,LOCATION
0,OAK HILL AVE,4067,UTAH ST,Maple,5,1,Fair,No,No,No,889983.173256,1.006871e+06,{000025D7-9880-4FEA-9418-289F5E506BE8},4067 UTAH ST,Yes,None,FALSE,Easement Tree,4067 UTAH ST
1,SWAN AVE,4427,SWAN AVE,Maple,18,1,Poor,No,No,No,568477.616427,1.016695e+06,{00004E14-7BFA-4BFA-AE16-B97C20D6C373},4427 SWAN AVE,Yes,None,FALSE,Easement Tree,4427 SWAN AVE
2,ENRIGHT AVE,5501,ENRIGHT AVE,Vacant,0,0,N/A,No,No,No,882794.969857,1.027355e+06,{00005BB3-56A7-4DE0-8830-2BD5FF695DDE},5501 ENRIGHT AVE,Yes,None,FALSE,Easement Tree,5501 ENRIGHT AVE
3,BLOW ST,3655,BLOW ST,Vacant,0,0,N/A,No,No,No,887255.794446,9.921068e+05,{000085FC-BD37-4D9D-BBF6-0E3287270D68},3655 BLOW ST,Yes,"Redbud, Eastern",FALSE,Easement Tree,3655 BLOW ST
4,COLEMAN AVE,1821,COLEMAN AVE,Vacant,0,0,N/A,No,No,No,899440.025369,1.025467e+06,{0000A65A-A225-4A9C-9FBA-213CA68E6D5F},1821 COLEMAN AVE,Yes,None,FALSE,Easement Tree,1821 COLEMAN AVE
5,CALVIN AVE,4861,CALVIN AVE,Vacant,0,0,N/A,No,No,No,893802.948940,1.037398e+06,{0000E6F6-F41D-4096-9601-F1B78B4B797C},4861 CALVIN AVE,Yes,None,FALSE,Easement Tree,4861 CALVIN AVE
6,KOSSUTH AVE,4219,KOSSUTH AVE,Sweetgum,13,1,Fair,No,Yes Sidewalk,No,896919.851370,1.033761e+06,{000186C2-F21A-44AB-B7FC-A837F3462448},4219 KOSSUTH AVE,Yes,None,FALSE,Easement Tree,4219 KOSSUTH AVE
7,BOWEN ST,5951,MARWINETTE AVE,Sweetgum,20,1,Good,No,No,No,886958.055463,9.962323e+05,{00019530-4CF3-48AB-9690-6E3493369A69},5951 MARWINETTE AVE,Yes,None,FALSE,Easement Tree,5951 MARWINETTE AVE
8,KOSSUTH AVE,4214,KOSSUTH AVE,Maple,13,1,Good,No,Yes Sidewalk,No,896909.714680,1.033718e+06,{0001F417-EDE2-4A3D-BBF8-4F5025E60F7A},4214 KOSSUTH AVE,Yes,None,FALSE,Easement Tree,4214 KOSSUTH AVE
9,HEGER CT,2603,HEGER CT,Dogwood,2,3,Good,No,No,No,887248.907190,1.011230e+06,{00020213-5B12-4B4D-92C9-D895455CE193},2603 HEGER CT,Yes,None,FALSE,Easement Tree,2603 HEGER CT


##Cleaning Housing Data

In [49]:
import pandas as pd



In [50]:
bdf = pd.read_csv('PrclSale.csv')
bdf['SaleDate'] = pd.to_datetime(bdf['SaleDate'])

bdf = bdf.loc[bdf['SaleDate'] >= '2010-01-01']
# bdf = bdf.dropna()
bdf = bdf.loc[bdf['SalePrice'] >= 10000]
bdf['block_median']= bdf.groupby('CityBlock')['SalePrice'].transform('median')
bdf = bdf.drop(labels = ['OwnerCode', 'SaleNum', 'AsrParcelId', 'LRMSSaleDate'], axis = 1)

bdf['block_num'] = (bdf['CityBlock'] +10000) * 100
bdf['block_num']= bdf['block_num'].astype(int)
#bdf.tail(50)
bdf.to_csv('median_block')

##Exploratory Data Analysis

In [51]:
bdf.head(50)

,CityBlock,Parcel,SaleType,SaleDate,SalePrice,block_median,block_num
2,15.00,90.0,0,2021-10-27,1000000.0,1000000.0,1001500
18,26.00,10.0,0,2022-01-04,1200000.0,1200000.0,1002600
19,26.00,40.0,54,2019-12-03,1087040.0,1200000.0,1002600
20,26.00,50.0,10,2019-09-19,2080000.0,1200000.0,1002600
22,27.00,80.0,54,2019-12-03,1693440.0,1693440.0,1002700
25,41.00,120.0,0,2022-10-19,550000.0,550000.0,1004100
26,48.00,40.0,40,2019-11-19,80000.0,80000.0,1004800
27,49.00,15.0,0,2022-02-28,1200000.0,1200000.0,1004900
37,67.00,61.0,0,2021-06-23,325000.0,462500.0,1006700
38,67.00,91.0,0,2022-02-01,600000.0,462500.0,1006700
